# 11. 병행성과 네트워크

지금까지의 프로그램은 한 장소(싱글 머신) 에서 한 번에 한 라인씩(순차적으로) 실행.  
여러 장소(**분산 컴퓨팅** 혹은 **네트워킹**)에서 동시에 여러 개의 일(**병행성**<sup>concurrency</sup>)할 수 있음.  

- **성능**<sup>performance</sup>
  느린 요소<sup>component</sup>를 기다리지 않고, 빠른 요소를 바쁘게 유지한다.
  
- **견고함**<sup>robustness</sup>
  하드웨어 및 소프트웨어의 장애를 피하기 위해 작업을 복제하여 여러 가지 안정적인 방식으로 운영한다.
  
- **간소화**<sup>simplicity</sup>
  복잡한 작업을 좀 더 이해하기 쉽고, 해결하기 쉬운 여러 작은 작업으로 분해한다.
  
- **커뮤티케이션**<sup>communication</sup>
  데이터(바이트)를 보내고 싶은 곳에 원격으로 전송하고, 다시 데이터를 수신받는다.

## 11.1. 병행성

컴퓨터 수행시 기다리는 두 가지 이유

- **I/O 바운드**
  대부분 이 경우에 해당. 컴퓨터의 CPU는 엄청나게 빠르다. 메모리 보다 몇 백배, 디스크나 네트워크보다 몇 천배.
- **CPU 바운드**
  과학이나 그래픽 작업과 같이 **엄청난 계산** 이 필요할 때 발생.
  
병행성과 관련있는 용어

- 동기<sup>synchronous</sup>
  한 줄의 장례 행렬처럼, 한 작업은 다른 작업을 따른다.
- 비동기<sup>asynchroonus</sup>
  사람들이 각기 다른 차를 타고 파티에 가는 것처럼, 작업들이 독립적이다.
  

- 정적 및 동적 페이지를 빠르게 제공해야 함
- 파일 업로드, 이미지 크기 조정 혹은 데이터베이스 쿼리 질의하는 시간
- 싱글 머신에서 다수의 작업을을 가능한 빠르게 처리하고 싶다면, 독립적으로 만들어야 함.

묘책은 서로 같이 일을 처리하기 위해 모든 작업을 가져온다.
하지만 병목<sup>bottleneck</sup>현상이 발생할 수 있음.

### 11.1.1 큐

- 리스트와 같음
- 먼저 들어온 순서대로 가져간다. **FIFO**<sup>First In First Out</sup>
- 메시지를 전달. 메시지는 모든 종류의 정보가 될 수 있음.
- 분산 작업 관리를 위한 큐의 경우는 **작업 큐**<sup>work queue, job queue, task queue</sup>라고 알려짐.

### 11.1.2 프로세스

싱글 머신에서 표준 라이브러리의 multiprocessing 모듈(10.3절 '프로그램과 프로세스'참조)은 Queue 함수 포함.

In [ ]:
import multiprocessing as mp

def washer(dishes, output):
    for dish in dishes:
        print('Washing', dish, 'dish')
        output.put(dish)

def dryer(input):
    while True:
        dish = input.get()
        print('Drying', dish, 'dish')
        input.task_done()       # 처리 완료 통보

if __name__ == "__main__":

    dish_queue = mp.JoinableQueue()         # 큐 생성
    dryer_proc = mp.Process(target=dryer, args=(dish_queue,))   # 큐에 있는 내용을 처리
    dryer_proc.daemon = True
    dryer_proc.start()

    dishes = ['salad', 'bread', 'entree', 'dessert']
    washer(dishes, dish_queue)
    dish_queue.join()       # 처리 완료때까지 대기


multiprocessing 모듈에는 다른 큐 타입도 있는데, 자세한 사항과 예제는 [문서](http://bit.ly/multi-docs) 참고.

### 11.1.3 스레드

- 한 프로세스 내에서 실행된다.
- 프로세스의 모든 자원에 접근 가능.
- threading 모듈


In [2]:
import threading

def do_this(what):
    whoami(what)

def whoami(what):
    print("Thread %s says: %s" % (threading.current_thread(), what))

if __name__ == "__main__":
    whoami("I'm the main program")
    for n in range(4):
        p = threading.Thread(target=do_this,
          args=("I'm function %s" % n,))
        p.start()


Thread <_MainThread(MainThread, started 12988)> says: I'm the main program
Thread <Thread(Thread-6, started 1160)> says: I'm function 0
Thread <Thread(Thread-7, started 15116)> says: I'm function 1
Thread <Thread(Thread-8, started 13372)> says: I'm function 2
Thread <Thread(Thread-9, started 13840)> says: I'm function 3


- 스레드는 전역 데이타가 관여하지 않을 떄 유용하고 안전하다.
- 전역 데이터 변경하기 위하여 때때로 스레드 사용하는 좋은 이유가 있다.
  여러 스레드를 사용하는 일반적인 이유는 일부 데이터 작업을 나누기 위해서.
- 데이터를 안전하게 공유하는 일반적인 방법은 **락**<sup>Lock</sup>(잠금)을 적용하는 것.

NOTE
> 파이썬의 스레드는 CPU 바운드 작업을 빠르게 처리하지 못한다. GIL<sup>Global Interpreter Lock</sup>이라는 표준 파이썬 시스템의 세부 구현사항 때문이다. GIL은 파이썬 인터프리터의 스레딩 문제를 피하기 위해 존재한다. 실제로 파이썬의 멀티 스레드 프로그램은 싱글 스레드 혹은 멀티 프로세스 버전의 프로그램보다 느릴 수 있다.

다음과 같이 파이썬을 사용할 것을 추천한다.
- I/O 바운드 문제 - 스레드 사용
- CPU 바운드 문제 - 프로세스, 네트워킹, 이벤트 사용

### 11.1.4 그린 스레드와 gevent

프로그램 속도가 느린 것을 피하는 방법
- 별도의 스레드, 프로세스를 실행.
- **이벤트 기반**<sup>event_based</sup>

이벤트 기반 프로그램
- 중앙 이벤트 루프 실행
- 엔진엑스 웹서버
- gevent

gevent  
  보통 명령 코드를 작성하고, 이 조각들을 **코루틴**<sup>coroutine</sup>으로 신비롭게 변환한다.  
  gevent는 블로킹<sup>blocking</sup> 대신 이러한 메커니즘을 사용하기 위해 파이썬의 socket과 같이 많은 표준 객체를 수정한다.  
  일부 테이터베이스 드라이버처럼 C로 자성된 파이썬의 확장 코드와 작동하지 않는다.  

  pip 설치  
  $ pip install gevent  
  
  gevent [웹사이트](https://pypi.python.org/pypi/gevent)에서 알맞은 버전의 파일을 내려받은 후 압축을 풀고, 다음과 같이 설치한다.  
  
  $ gevetn-1.1b$ python setup.py install
  
  


In [ ]:
import gevent
from gevent import socket
hosts = ['www.crappytaxidermy.com', 'www.walterpottertaxidermy.com',
    'www.antique-taxidermy.com']
jobs = [gevent.spawn(gevent.socket.gethostbyname, host) for host in hosts]
gevent.joinall(jobs, timeout=5)  # 생성된 모든 작업이 끝날때 까지 대기. 호스트네임에 대한 IP 주소를 한번에 얻는다.
for job in jobs:
    print(job.value)
    

gevent.spawn()은 각각의 gevent.socket.gethostbyname(host)를 실행하기 위해 greelet(그린 스레드<sup>green thread</sup>혹은 마이크로 스레드<sup>microthread</sup>라고 알려져 있음)을 생성.  

greenlet과 일반적인 스레드의 차이점은 블록<sup>block</sup>을 하지 않음. 만약 한 스레드에서 무슨 일이 발생하여 블록되었다면 gevent는 다른 하나의 greenlet으로 바꾼다.

gevent 버전의 socket대신, 기억하기 쉬운 이름의 **몽키-패치**<sup>monkey-patch</sup>함수를 쓸 수 있다.  
이 함수는 gevent 버전의 모듈을 호출하지 않고, greenlet을 사용하기 위해 socket과 같은 표준 모듈을 수정한다.
이것은 gevent에 적용하고 싶은 작업이 있을 때 유용하다. 심지어 gevent에 접근할 수 없는 코드에도 적용할 수 있다.



In [ ]:
from gevent import monkey
monkey.patch_socket()

프로그램과 표준라이브러리에서도 소켓이 호출되는 모든 곳에 gevent소켓을 사용하겠다는 뜻이다. 파이썬 코드에서만 작동한다.

In [ ]:
import gevent
from gevent import monkey; monkey.patch_all()
import socket
hosts = ['www.crappytaxidermy.com', 'www.walterpottertaxidermy.com',
    'www.antique-taxidermy.com']
jobs = [gevent.spawn(socket.gethostbyname, host) for host in hosts]
gevent.joinall(jobs, timeout=5)
for job in jobs:
    print(job.value)


gevent를 사용하면 잠재적 위험이 있다. 많은 일을 처리해야 하는 코드는 여전히 느리다.
놓여있는 상황과 지시에 따라 gevent를 잘 활용하면 된다.

NOTE
> [tomado](http://www.tornadoweb.org)와 [gunicom](http://gunicorn.org)이라는 인기 있는 이벤트 기반의 두 프레임워크가 있다. 이들은 저수준의 이벤트 처리와 빠른 웹 서버 모두를 제공한다. 아파치와 같은 전통적인 웹 서버 없이 빠른 웹사이트를 구축하기 위해 이들을 접해볼 가치가 있다.

### 11.1.5 twisted

- [twisted](http://twistedmatrix.com/trac) 는 비동기식 이벤트 기반 네트워킹 프레임워크
- 콜백<sup>callback</sup> 디자인

파이썬 2 인스롤러로 twisted 를 설치
 $ pip2 install twisted

In [ ]:
from twisted.internet import protocol, reactor

class Knock(protocol.Protocol):
    def dataReceived(self, data):
        print 'Client:', data
        if data.startswith("Knock knock"):
            response = "Who's there?"
        else:
            response = data + " who?"
        print 'Server:', response
        self.transport.write(response)

class KnockFactory(protocol.Factory):
    def buildProtocol(self, addr):
        return Knock()

reactor.listenTCP(8000, KnockFactory())
reactor.run()


In [ ]:
from twisted.internet import reactor, protocol

class KnockClient(protocol.Protocol):
    def connectionMade(self):
        self.transport.write("Knock knock")

    def dataReceived(self, data):
        if data.startswith("Who's there?"):
            response = "Disappearing client"
            self.transport.write(response)
        else:
            self.transport.loseConnection()
            reactor.stop()

class KnockFactory(protocol.ClientFactory):
    protocol = KnockClient

def main():
    f = KnockFactory()
    reactor.connectTCP("localhost", 8000, f)
    reactor.run()

if __name__ == '__main__':
    main()


먼저 서버를 실행

 $ python2 knock_server.py
 
그리고 나서 클라이언트를 실행.

 $ python2 knock_client.py
 
 
 Client: knock knock  
 Server: Who's there?  
 Client: Disappearing client  
 Server: Disappearing Client who?  

### 11.1.6 asyncio

- 많은 패키지는 자신만의 이벤트 루프를 가지고 있다.
- '비동기 입출력 지원 재정리:asyncio 모듈<sup>Asynchronous IO Support Rebooted: the "asyncio" Module</sup>'(코드 이름 [튤립](http://bit.ly/pep-3156)<sup>Tulip</sup>)을 제안.
- 현재 twisted와 gevent 그리고 다른 비동기 메서드와 호환될 수 있는 일반적인 이벤트 루프를 제공.
- asyncio 모듈의 다양한 [예제](https://docs.python.org/3/library/asyncio.html)가 수록.

### 11.1.7 Redis

- 싱글박수(하나의 머신)와 멀티박스 병행성 사이를 연결해주는 브리지(다리)
- 예제 실행을 위한 Redis 서버, Redis 파이썬 모듈 필요
- Redis에 대한 설명은 8.5.3절 'Redis'를 참조.
- 큐를 만들 수 있는 빠른 방법은 Redis 의 리스트.

In [ ]:
import redis
conn = redis.Redis()
print('Washer is starting')
dishes = ['salad', 'bread', 'entree', 'dessert']
for dish in dishes:
    msg = dish.encode('utf-8')
    conn.rpush('dishes', msg)
    print('Washed', num)
conn.rpush('dishes', 'quit')
print('Washer is done')


In [ ]:
import redis
conn = redis.Redis()
print('Dryer is starting')
while True:
    msg = conn.blpop('dishes')
    if not msg:
        break
    val = msg[1].decode('utf-8')
    if val == 'quit':
        break
    print('Dried', val)
print('Dishes are dried')


In [ ]:
def dryer():
    import redis
    import os
    import time
    conn = redis.Redis()
    pid = os.getpid()
    timeout = 20
    print('Dryer process %s is starting' % pid)
    while True:
        msg = conn.blpop('dishes', timeout)
        if not msg:
            break
        val = msg[1].decode('utf-8')
        if val == 'quit':
            break
        print('%s: dried %s' % (pid, val))
        time.sleep(0.1)
    print('Dryer process %s is done' % pid)

import multiprocessing
DRYERS=3
for num in range(DRYERS):
    p = multiprocessing.Process(target=dryer)
    p.start()

    

## 11.2 네트워크

### 11.2.1 패턴

- 요청-응답 패턴(클라이언트-서버)
  동기적. 클라이언트는 서버의 응답이 올 때 까지 기다린다.
  
- 푸시<sub>push</sub> 또는 팬아웃<sub>fanout</sub>
  데이터를 프로세스 풀<sub>pool</sub>에 있는 사용 가능한 워커로 전송. 로드밸런서 뒤에는 웹서버가 있음.
- 풀<sub>pull</sub> 또는 펜인<sub>fanin</sub>.
  하나이상의 소스로부터 데이터를 받는다. 멀리 프로세스에서 텍스트 메시지를 받아서, 하나의 로그 파일에 작성하는 로거<sub>logger</sub>가 있음.
- 발행-구독<sub>publish-subscribe</sub>(pub-sub)하는 라디오나 텔레비전의 방송과 유사하다.

### 11.2.2 발행-구독 모델

- 큐가 아닌 브로드캐스트<sub>broadcast</sub>.
- 하나 이상의 프로세스가 메시지를 발행.
- 각 구독자 프로세느는 수신하고자 하는 메시지의 타입을 표시.
- 각 메시지의 복사본은 타입과 일치하는 구독자에게 전송.
- 주어진 메시지는 한 번 또는 여러 번 처리되거나, 아예 처리되지 않을 수도 있음.
- 각 발행자는 단지 브로드캐스팅<sub>broadcasting</sub>만 할 뿐, 누가 구독하는지 알지 못함.

### Redis

발행-구독 시스템을 빠르게 구현 가능.  
구독자는 토픽, 값과 함께 메시지를 전달하고 구독자는 수신받고자 하는 토픽을 말한다.

In [ ]:
#redis_pub.py

import redis
import random

conn = redis.Redis()
cats = ['siamese', 'persian', 'maine coon', 'norwegian forest']
hats = ['stovepipe', 'bowler', 'tam-o-shanter', 'fedora']
for msg in range(10):
    cat = random.choice(cats)  # 각 토픽은 고양이 품종
    hat = random.choice(hats)  # 메시지 타입
    print('Publish: %s wears a %s' % (cat, hat))
    conn.publish(cat, hat)


In [ ]:
#redis_sub.py

import redis
conn = redis.Redis()

topics = ['maine coon', 'persian']
sub = conn.pubsub()
sub.subscribe(topics)
for msg in sub.listen():
    if msg['type'] == 'message':
        cat = msg['channel']
        hat = msg['data']
        print('Subscribe: %s wears a %s' % (cat, hat))


### ZeroMQ

중앙 서버가 없으므로, 각 발행자는 모든 구독자에 메시지를 전달.

In [ ]:
#Zmq_pub.py

import zmq
import random
import time
host = '*'
port = 6789
ctx = zmq.Context()
pub = ctx.socket(zmq.PUB)
pub.bind('tcp://%s:%s' % (host, port))
cats = ['siamese', 'persian', 'maine coon', 'norwegian forest']
hats = ['stovepipe', 'bowler', 'tam-o-shanter', 'fedora']
time.sleep(1)
for msg in range(10):
    cat = random.choice(cats)
    cat_bytes = cat.encode('utf-8')
    hat = random.choice(hats)
    hat_bytes = hat.encode('utf-8')
    print('Publish: %s wears a %s' % (cat, hat))
    pub.send_multipart([cat_bytes, hat_bytes])


In [ ]:
#zmq_sub.py

import zmq
host = '127.0.0.1'
port = 6789
ctx = zmq.Context()
sub = ctx.socket(zmq.SUB)
sub.connect('tcp://%s:%s' % (host, port))
topics = ['maine coon', 'persian']
for topic in topics:
    sub.setsockopt(zmq.SUBSCRIBE, topic.encode('utf-8'))
while True:
    cat_bytes, hat_bytes = sub.recv_multipart()
    cat = cat_bytes.decode('utf-8')
    hat = hat_bytes.decode('utf-8')
    print('Subscribe: %s wears a %s' % (cat, hat))


구독자 먼저 실행한다.

$ python zmq_sub.py

그 다음에 발행자를 실행한다.

$ python zmq_pub.py

### 기타 발행-구독 도구

- RabbitMQ
  메시징 보르커로 잘 알려져 있다. pika 는 RabbitMQ를 위한 파이썬 API이다. pika [문서](http://pika.readthdocs.org)와 publish/subscribe [튜토리얼](http://bit.ly/pub-sub-tut)을 참고한다.
  
- pypi.python.org
  오른쪽 상단의 검색창에 pubsub를 입력하여 [pypubsub](http://pubsub.sourceforge.net)와 같은 패키지를 찾아본다.
- pubsubhubbub
  이 감미로운 [프로토콜](https://github.com/pubsubhubbub)은 구독자가 발행자와 함께 콜백을 등록할 수 있게 해준다.

### 11.2.3 TCP/IP

인터넷은 커넥션을 맺고, 데이터를 교환하고, 커넥션을 종료하고, 타임아웃을 처리하는 등의 방법에 대한 규칙에 의거한다. 이것을 프로토콜<sub>protocol</sub>이라고 하며, 계층<sub>layer</sub>으로 정렬되어 있다.

IP계증에는 네트워크 위치 사이에서 바이트를 이동하는 방벙을 기술하는 다음 두 가지 프로토콜이 있다.

- UDP<sub>User Datagram Protocol(사용자 데이터그램 프로토콜)
  이 프로토콜은 짧은 데이터 교환에 사용된다. 데이터그램은 엽서의 짧은 글처럼, 한 단위<sub>single burst</sub>로 전송되는 작은 메시지다.
- TCP<sub>Transmission COntrol Protocol(전송 제어 프로토콜)
  이 프로토콜은 수명이 긴 커넥션에 사용된다. TCP는 바이트 스트림이 중목 없이 순서대로 도착하는 것을 보장한다.
  
UDP 메시지는 응답 메시지<sub>ACK</sub>가 없다. 

TCP는 송신자와 수신자 사이의 커넥션을 보장하기 위해 핸드셰이크<sub>handshake</sub>를 설정한다.

웹과 데이터베이스 서버 등 우리가 소통하는 대부분의 인터넷은 IP 프로토콜의 맨 위에서 실행되고 있는 TCP프로토콜(TCP/IP).

### 11.2.4 소켓



In [ ]:
#udp_server.py

from datetime import datetime
import socket

server_address = ('localhost', 6789) # (주소, 포트)의 튜플
max_size = 4096

print('Starting the server at', datetime.now())
print('Waiting for a client to call.')
server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # AF_INET: 인터넷(IP), SOCK_DGRAM : 데이터그램 송신(UDP)
server.bind(server_address)

data, client = server.recvfrom(max_size) # 데이터그램 wait, client는 클라이언트의 주소와 포트 정보

print('At', datetime.now(), client, 'said', data)
server.sendto(b'Are you talking to me?', client)
server.close()


In [ ]:
#udp_client.py

from datetime import datetime
import socket

server_address = ('localhost', 6789)
max_size = 4096

print('Starting the server at', datetime.now())
print('Waiting for a client to call.')
server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server.bind(server_address)

data, client = server.recvfrom(max_size)

print('At', datetime.now(), client, 'said', data)
server.sendto(b'Are you talking to me?', client)
server.close()


서버를 실행한다.
$ python udp_server.py

클라이언트를 실행한다.
$ python udp_client.py

NOTE
> UDP는 한 청크<sub>chunk</sub>에 데이터를 보낸다. 데이터 전송을 보장하지 않는다. UDP를 통해 여러 메시지를 보낼 경우, 순서 없이 도착하거나 모두 도착하지 않을 수 있다. UDP는 빠르고, 가볍고, 신뢰할 수 었고, 비연결형<sub>connectionless</sub>이다.

TCP를 선호하는 이유
- TCP는 웹과 같은 수명이 긴 커넥션에 사용.
- 데이타를 보낸 순서대로 전달.
- 전송에 문제가 생기면 다시 보낸다.


In [ ]:
#tch_client.py

import socket
from datetime import datetime

address = ('localhost', 6789)
max_size = 1000

print('Starting the client at', datetime.now())
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM) # TCP를 얻기 위해서 SOCK_STREAM사용.
client.connect(address) # 스트림 설정
client.sendall(b'Hey!')
data = client.recv(max_size)
print('At', datetime.now(), 'someone replied', data)
client.close()


In [ ]:
#tcp_server.py

from datetime import datetime
import socket

address = ('localhost', 6789)
max_size = 1000

print('Starting the server at', datetime.now())
print('Waiting for a client to call.')
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # TCP를 얻기 위해서 SOCK_STREAM사용.
server.bind(address)
server.listen(5)# 대기 중인 커넥션의 최대 수 지정.

client, addr = server.accept()
data = client.recv(max_size) # 최대 허용 메시지 길이 제한

print('At', datetime.now(), client, 'said', data)
client.sendall(b'Are you talking to me?')
client.close()
server.close()


TCP, UDP 특징
- UDP는 메시지를 전송하는 데 크기의 제한이 있다. 그리고 메시지가 목적지까지 도달하는 것을 보장하지 않는다.
- TCP는 메시지가 아닌 바이트 스트림을 전송한다. 시스템의 각 송수신 호출에서 얼마나 많은 바이트가 전달되는지 알 수 없다.
- TCP를 통해 전체 메시지를 전달하기 위해 세그먼트<sub>segment</sub>로부터 전체 메시지를 재구성하기 위한 몇가지 추가 정보가 필요하다.(고정 메시지 크기(바이트), 메시지 전체 크기, 구분자).
- 메시지는 유니코드 텍스트 문자열이 아닌 바이트이기 때문에 파이썬 바이트 타입을 사용해야 한다. 유니코드와 바이트 타입에 대한 자세한 사항은 7장을 참조한다.

파이썬 소켓 프로그램이에 관한 좀 더 자세한 사항은 [Socket Programming HOWTO](http://bit.ly/socket-howto)를 참고한다.

### 11.2.5 ZeroMQ

ZeroMQ는 라이브러리이다.

하는일
- 전체 메시지 교환
- 커넥션 재시도
- 송신자와 수신자 사이에 전송 타이밍이 맞지 않는 경우 데이터 보존을 위한 버퍼 사용

[온라인 가이드](http://zguide.zeromq.org) 참조.
"ZeroMQ: MEssageing for Many Application"(Pieter Hintjens 저, 오라일리) 의 책. 관련 파이썬 [예제](http://bit.ly/zeromq-py)

ZeroMQ 소켓 타입
- REQ(동기 요청<sub>synchronous request</sub>)
- REP(동기 응답<sub>synchronous reply</sub>)
- DEALER(비동기 요청<sub>asynchronous request</sub>)
- ROUTER(비동기 응답<sub>asynchronous reply</sub>)
- PUB(발행<sub>publish</sub>)
- SUB(구독<sub>subscribe</sub>)
- PUSH(팬아웃<sub>fanout</sub>)
- PULL(팬인<sub>fanin</sub>)

ZeroMQ 라이브러리 설치.
$ pip install pyzmg

In [ ]:
#zmq_server.py

import zmq

host = '127.0.0.1'
port = 6789
context = zmq.Context()
server = context.socket(zmq.REP)
server.bind("tcp://%s:%s" % (host, port))
while True:
    #  클라이언트에서 다음 요청을 기다린다.
    request_bytes = server.recv()
    request_str = request_bytes.decode('utf-8')
    print("That voice in my head says: %s" % request_str)
    reply_str = "Stop saying: %s" % request_str
    reply_bytes = bytes(reply_str, 'utf-8')
    server.send(reply_bytes)


In [ ]:
#zmq_client.py

import zmq

host = '127.0.0.1'
port = 6789
context = zmq.Context()
client = context.socket(zmq.REQ)
client.connect("tcp://%s:%s" % (host, port))
for num in range(1, 6):
    request_str = "message #%s" % num
    request_bytes = request_str.encode('utf-8')
    client.send(request_bytes)
    reply_bytes = client.recv()
    reply_str = reply_bytes.decode('utf-8')
    print("Sent %s, received %s" % (request_str, reply_str))


서버를 백그라운드로 실행

$ python zmq_server.py &

클라이언트를 실행

$ python zmq_client.py


- 메시지는 바이트 문자열로 전송해야 함.
- UTF-8 포맷으로 텍스트 문자열을 인코딩.
- 메시지를 바이트로 변환하면 모든 종류의 메시지를 전송할 수 있다.
- encode(), decode()로 문자열을 바이트로, 바이트를 문자열로 변환
- 메시지가 다른 데이터 타입이라면 [MessagePack](http://msgpack.org) 과 같은 라이브러리 사용.

기본적인 요청-응답<sup>REQ-REP</sup>패턴은 일부 복잡한 커뮤니케이션 패턴을 허용한다.
- 메시지를 특정 한계까지, 가능한 만큼 버퍼에 넣는다.
- Q : 큐<sup>Queue</sup>, M은 메시지<sup>Message</sup>, Zero는 브로커<sup>broker</sup>가 필요없다는 의미.
- 중앙 브로커가 없지만, 필요한 경우 브로커를 만들수있음.(EEALER, ROUTER소켓 이용)
  1. 여러개의 REQ소켓은 하나의 ROUTER소켓에 연결된다. ROUTER 소켓을 통과해서 DEALER 소켓으로 전달된다.
  2. DEALER 소켓은 연결된 어떤 REP소켓에 접근한다.
  3. 이것은 웹 서버 팜<sup>web server farm</sup>앞의 프록시 서버<sup>proxy server</sup>에 접근하는 다수의 브라우저와 유사.
  

마지막으로 ZeroMQ의 중요한 특징은 소켓이 생성될 때 소켓의 커넥션 타입을 바꿔서 스케일업<sup>scale up</sup>과 스케일 다운<sup>scale down</sup>을 한다.
- tcp : 하나 이상의 머신에서 프로세스 간 통신
- ipc<sup>Inter-Process Communication</sup> : 하나의 머신에서 프로세스 간 통신
- inproc<sup>IN-PROCess(inter_thread) Communication</sup> : 한 프로세스에서 스레드 간 통신

inproc 은 락<sub>lock</sub>없이 스레드 간 데이터를 전달하는 방법이다. 

NOTE
>ZeroMQ는 파이썬이 지원하는 유일한 메시지 전달<sup>message-passing</sup>라이브러리가 아니다. 메시지 전달은 네트워킹에서 가장 인기 있는 아이디어 중 하나다. 그리고 파이썬은 다른 언어와 함께 네트워크 라이브러리를 지원하고 있다. 9.2.6절에서 본 '아파치'웹 서버의 아파치 프로젝트 또한 간단한 텍스트 지향의 [STOMP](http://stomp.github.io/implementations.html)<sup>Simple (or Streaming) Text Orientated Messaging Protocol</sup> 프로토콜을 사용한 몇몇 파이썬 인터페이스가 포함된 [ActiveMQ](https://activemq.apache.org)를 관리하고 있다. [RabbitMQ](http://www.rabbitmq.com)역시 유용한 파이썬 [튜토리얼](http://bit.ly/rabbitmq-tut)를 제공하며, 매우 인기 있다.
   


### 11.2.6 Scapy

- 패킷을 분석하기 위한 유용한 파이썬 도구.
- C 프로그램보다 코드를 작성하고 디버깅하는 것이 휠씬 쉽다.
- 실제로 패킷을 만들고, 분석하기 위한 작은 언어다.

몇 가지 예제 코드를 포함하지 않은 이유
- scapy는 아직 파이썬3으로 옮겨지지 않았다. 
- 입문하기 위한 절차가 까다롭다. 설치하는 방법은 [웹 페이지](http://bit.ly/scapy-install)를 참고한다.

관심이 있다면 [웹 문서](http://bit.ly/scapy-docs)에서 예제를 살펴본다. 

### 11.2.7 인터넷 서비스

파이썬은 광범위한 네트워킹 도구 세트를 제공.
전반적인 내용은 파이썬 공식 [문서](http://bit.ly/py-internet)참고.

### DNS

분산된 데이터베이스를 통해 IP주소를 이름으로 바꾸거나, 그 반대를 수행하는 아주 중요한 인터넷 서비스.

저수준의 socket 모듈.  
gethostbyname()은 도메인 이름에 대한 IP 주소를 반환.  
gethostbyname_ex()은 이름, 또 다른 이름의 리스트, 주소의 리스트를 반환.

In [2]:
import socket
socket.gethostbyname('www.crappytaxidermy.com')

'66.6.44.4'

In [4]:
socket.gethostbyname_ex('www.crappytaxidermy.com')

('crappytaxidermy.com', ['www.crappytaxidermy.com'], ['66.6.44.4'])

getaddrinfo() 는  IP주소를 검색. 또한 소켓을 생성하고 연결하기 위한 충분한 정보를 반환.

In [6]:
socket.getaddrinfo('www.crappytaxidermy.com',80) # 첫번쨰는 UDP, 두번쨰는 TCP

[(<AddressFamily.AF_INET: 2>, 0, 0, '', ('66.6.44.4', 80))]

일부 TCP와 UDP포트 [번호](http://bit.ly/tcp-udp-ports)는 IANA<sup>Internet Assigned Numbers Authority</sup>(인터넷 할당 번호 관리기관)에 의해 특정 서비스에 예약되어 있고 서비스 이름과 연결되어 있다. HTTP 의 이름은 http, TCP 80 포트로 할당.

In [8]:
import socket
socket.getservbyname('http')

80

### 파이썬 이메일 모듈

- smtplib 모듈:SMTP<SUP>Simple Mail Transfer Protocol</sup>를 통해 이메일 전송하기
- email 모듈:이메일 생성 및 파싱하기
- poplib 모듈:POP3<sup>Post Office Protocol 3</sup>를 통해 이메일 읽기
- imaplib 모듈:IMAP<sup>Internet Message Access Protocol</sup>을 통해 이메일 읽기

파이썬 공식 문서는 위 모든 라이브러리에 대한 샘플 [코드](http://bit.ly/py-email)를 제공한다.
파이썬으로 SMTP서버를 만들고 싶다면 smtpd [모듈](http://bit.ly/py-smtpd)을 참고한다.
[Lamson](https://github.com/zedshaw/lamson)이라는 순수 파이썬 SMTP서버는 이메일 메시지를 데이터베이스에 저장할 수 있을뿐만 아니라 스팸 메일도 차단할 수 있다.

### 기타 프로토콜

표준 ftplib 모듈은 FTP<sup>File Transfer Protocol</sup>를 사용하여 바이트를 전송할 수 있다.
웹 [문서](http://bit.ly/py-internet)참고

### 11.2.8 웹 서비스와 API

가장 쉬운 API는 일반 텍스트나 HTML보다 JSON이나 XML같은 구조화 된 포맷의 데이터를 제공하는 웹 인터페이스다.  
API는 최소한으로 혹은 제대로 갖춘 RESTful API(9.3.2절 '웹 API와 REST'참조)일 수도 있지만, 불규칙적인 데이터를 위한 또 다른 인터페이스를 제공한다.  
API는 트위터, 페이스북, 링크드인과 같은 잘 알려진 소셜 미디어 사이트의 채굴작업에 특히 유용하다.

흥미로운 서비스API
- [뉴욕타임즈](http://developer.nytimes.com)
- [유투브](http://gdata.youtube.com/demo/index.html)
- [트위터](https://dev.twitter.com/overview/api)
- [페이스북](http://developer.facebook.com/tools)
- [Weather Underground](http://www.wunderground.com/weather/api)
- [Marvel Comics](http://developer.marvel.com)

### 11.2.9 원격 프로세싱

### 원격 프로시저 호출

RPC<sup>Remote Procedure Call</sup>(원격 프로시저 호출)은 네트워크를 통해 원격에 있는 머신을 실행.

RPC 클라이언트
1. 함수의 인자를 바이트로 변환한다.(마샬링<sup>marshalling</sup> 또는 직렬화<sup>serializing</sup>또는 그냥 인코딩이라고 부른다.)
2. 인코딩된 바이트를 원격 머신으로 전송한다.

원격 머신
1. 인코딩된 요청 바이트를 수신한다.
2. 바이트르 수신한 후, RPC 클라이언트는 다시 원래의 데이터 구조(혹은 다른 두 머신 사이의 하드웨어와 소프트웨어가 서로 다른 경우 그에 맞는 구조)로 바이트를 디코딩한다.
3. 클라이언트는 디코딩된 데이터와 함께 로컬 함수를 찾아서 호출한다.
4. 함수 결과를 인코딩한다.
5. 클라이언트는 인코딩된 바이트를 다시 호출자에 전송한다.

마지막으로 호출자의 머신에서 반환된 바이트 값을 디코딩한다.

In [ ]:
#xmlrpc_server.py

from xmlrpc.server import SimpleXMLRPCServer

def double(num):
    return num * 2

server = SimpleXMLRPCServer(("localhost", 6789))
server.register_function(double, "double")
server.serve_forever()

In [ ]:
#xmlrpc_client.py

import xmlrpc.client

proxy = xmlrpc.client.ServerProxy("http://localhost:6789/")
num = 7
result = proxy.double(num)
print("Double %s is %s" % (num, result))


XML을 제외한 일반적인 인코딩에는  JSON, 프로토콜 버퍼<sup>Protocol Buffer</sup>, 메시지팩<sup>MessagePack</sup>이 있다.

In [ ]:
#msppack_server.py

from msgpackrpc import Server, Address

class Services():
    def double(self, num):
        return num * 2

server = Server(Services())
server.listen(Address("localhost", 6789))
server.start()


In [ ]:
#msppack_client.py

from msgpackrpc import Client, Address

client = Client(Address("localhost", 6789))
num = 8
result =  client.call('double', num)
print("Double %s is %s" % (num, result))


### fabric

원격 또는 로컬 명령이나 파일 업로드 및 다운로드를 sudo 권한의 사용자로 실행한다.
원격 머신의 프로그램을 실행하기 위해 fabric패키지는 SSH<sup>Secure SHell</sup>(텔넷을 대체하는 암호화된 텍스트 프로토콜)를 사용한다.
 
설치  
$ pip2 install fabric

In [ ]:
# fab1.py

def iso():
    from datetime import date
    print(date.today().isoformat())


$ fab -f fab1.py -H localhost iso

-f fab1.py는 fabfile.py 대신 fab1.py로 지정.   
-H localhost 옵션은 명령을 로컬 머신에서 실행.  
iso는 fab 파일에서 실행할 함수 이름.
[웹문서](http://docs.fabfile.org)에서 참고

SSH 서버.
유닉스 계열의 시스템에서는 SSH 서버는 sshd.  
service sshd status 명령으로 sshd가 구동되는지 확인.

맥에서는 시스템 환경 설정<sup>System Preferences</sup>을 열고, 공유 탭<sup>Sharing</sup>을 클릭한 후 원격 로그인<sup>Remote Login</sup>을 체크한다.

윈도우는 내부적으로 SSH지원하지 않는다. 가장 좋은 방법은 [putty](http://bit.ly/putty-ssh)를 설치하는 것.

### SaltStack

[SaltStack](http://www.saltstack.com)은 원격 실행의 구현으로 시작했지만, 본격적인 시스템 관리 플랫폼으로 성장했다.
SSH가 아닌 ZeroMQ를 기반으로 한 SaltStack은 수천 개의 서버를 확장할 수 있다.

### 11.2.10 빅데이타와 맵리듀스

- 네트워크가 연결된 머신에서 데이터를 분산하여 분석하는 것이 개별적인 머신에서 수행하는 것이 빠르다.
- 큐 작업과 비슷.
- 구글에서 논문으로 발표한 후, 야후에서 하둡<sup>Hadoop</sup>이라는 자바 기반의 오픈소스 패키지를 개발.
- 빅데이터는 '아주 큰 데이터를 머신에 맞게<sup>data too big to fit on my machine</sup>라는 문구의 의미가 담겨있다.
- 하둡의 리아벌인 [스파크](http://bit.ly/about-spark)<sup>Spark</sup>는 하둡보다 10배에서 100배 빠르게 실행하도록 설계되었다.


### 11.2.11 클라우드

- 서버 구매
- 볼트질
- 소프트웨어 설치
- 시스템 죽지 않도록 노력
- 보안 문제 

복잡한 분산 시스템을 구축하기 위해 많은 작업과 다양한 도구 세트가 필요.

분산시스템을 구축하는 대신 클라우드의 서버를 임대.  
유지보수를 다른 누군가의 문제로 돌릴 수 있고 제공하고 싶은 서비스에만 집중.  
서버를 탄력적으로 운영할 수 있다.

### 구글

구글 클라우드 메인 [페이지](http://cloud.google.com)
- App Engine
  flask와 django같은 파이썬 도구를 포함하는 높은 수준의 플랫폼
- Compute Engine
  대규모 분산 컴퓨팅 작업을 위한 가상 머신 클러스터를 생성
- Cloud Storage
  오브젝트(객체)스토리지(오브젝트는 파일이지만, 디렉터리 구조는 아니다.)
- Cloud Datestore
  대규모 NoSQL 데이터베이스
- Cloud SQL
  대규모 SQL 데이터베이스
- Cloud Endpoints
  애플리케이션 접근(Restful)
- BigQuery
  하둡 계열의 빅데이터

### 아마존

- Elastic Beanstalk
  높은 수준의 애플리케이션 플랫폼
- EC2<sup>Elastic Compute</sup>
  분산 컴퓨팅
- S3<sup>Simple Storage Service</sup>
  오브젝트 스토리지
- RDS
  관계형 데이터베이스(MySQL,PostgreSQL, Oracle, MSSQL)
- DynamoDB
  NoSQL 데이터베이스
- Redshift
  데이터 웨어하우스
- EMR
  하둡
  
AWS 서비스에 대한 자세한 내용은 Amazon Python [SDK](http://bit.ly/aws-py-sdk)를 내려받아서 도움말을 참고.
공식 파이썬 AWS라이브러리는 [boto](http://docs.pythonboto.org)다.

### 오픈스택

- 미국 항공우주국(NASA)와 파트너쉽을 맺었다.
- 공개, 비공개, 하이브리드 클라우드를 구축하기 위해 자유롭게 사용할 수 있는 오픈소스 플랫폼.
- 메인 API는 RESTful

다음은 표준 서비스의 일부이다.
- keystone
  인증(예:사용자/비밀번호)과 권한(기능), 서비스 디스커버리를 제공하는 신원서비스
- Nova
  네트워크에 연결된 서버를 통해 작업을 배포하는 컴퓨팅 서비스
- Swift
  아마존 S3와 같은 오브젝트 스토리지. Rackspace 의 클라우드 파일 서비스에서 사용한다.
- Glance
  중간수준의 이미지 스토리지 서비스
- Cinder
  저수준의 블록 스토리지 서비스
- Horizon
  서비스에 대한 웹 기반의 대시보드
- Neutron
  네트워크 관리 서비스
- Heat
  오케스트레이션<sup>Orchestration</sup>(멀티클라우드)서비스
- Ceilometer
  텔레메트리<sup>Telemetry</sup>(측정, 모니터링, 미터링)서비스
  
설치의 가장 빠른 방법은 [DevStack](http://devstack.org)을 사용하는 것.